# Ford Stock Data

In [63]:
import pandas as pd
import os
#%pip install pytrends
import pytrends
from pytrends.request import TrendReq
#pip install pageviewapi
import pageviewapi
#%pip install yfinance
import yfinance as yf

In [64]:
def Big_scraper(kw_list_1, kw_list_2, ticker, start,end):
    
    """
    Description:
    ------------
    
    The function initially grabs historical, indexed, hourly data for when the keyword 
    was searched most as shown on Google Trends' Interest Over Time section.
    It then cleans the data to show daily hits on the keyword in Google news.

    
    input:
    -----
    kw_list_1: List of up to 5 key words that will be scraped from google trends for the dates given.
             Here, the scraping will pull the total posted items in google news that contains
             one of the key words.
    
    kw_list_2: List of wikipedia article titles (unlimited length) that will pull the amount of
            views the article recieved each day. 

    ticker: the ticker abriviation of the desired stock. Must be netered in as an all capitalized string 
    example Apple Inc. woud be "AAPL"
             
    start: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end: the end of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
             
    return:
    -------
    
    combined: a dataframe containing the sum of the daily keyword hits in google news (key words labeled _x),
    
    data frame cointaing stock info including open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day,
    
    and the sum of how many times key word wikipedia pages were viewed in a day (key words labeled _y)
    """
    
    year_s = int(start[0:4])
    month_s = int(start[4:6])
    day_s = int(start[6:8])
    year_e = int(end[0:4])
    month_e = int(end[4:6])
    day_e = int(end[6:8])
    
    starter = pd.to_datetime(f"'{year_s}-{month_s}-{day_s}'")
    ender = pd.to_datetime(f"'{year_e}-{month_e}-{day_e}'")
    
    
    pytrends = TrendReq(hl='en-US', tz=360, retries=10)
    jeff = pytrends.get_historical_interest(kw_list_1, \
                                 year_start = year_s, month_start = month_s, day_start = day_s, hour_start = 1, \
                                 year_end = year_e, month_end = month_e, day_end = day_e, hour_end = 23, \
                                 cat = 0, geo = '', gprop = 'news', sleep = 60)
    
    jeff = jeff.iloc[:, 0:-1] # eliminates the isPartial Column
    jeff = jeff.reset_index().drop_duplicates(subset = "date") #removing duplicates from the index
    jeff = jeff.groupby(pd.Grouper(key="date", freq="D")).mean() # coverts to the mean of daily scores

    dow = yf.Ticker("^DJI")
    dow_h = dow.history(start=starter, end=ender)
    dow_h = pd.DataFrame(dow_h)
    dow_names = {"Open":"dow_open","Close":"dow_close","Low": "dow_low",
    'High':'dow_high','Volume':'dow_vol'}
    dow_h=dow_h.rename(dow_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    nas = yf.Ticker("^IXIC")
    nas_h = nas.history(start=starter, end=ender)
    nas_h = pd.DataFrame(nas_h)
    nas_names = {"Open":"nas_open", "Close":"nas_close", "Low": "nas_low",
    'High':'nas_high','Volume':'nas_vol'}
    nas_h=nas_h.rename(nas_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    market = dow_h.merge(nas_h,left_index=True, right_index=True, how="left")
    
    tick = yf.Ticker(ticker)
    hist = tick.history(start=starter, end=ender)
    hist = pd.DataFrame(hist)
    
    combined = jeff.merge(hist, left_index=True, right_index=True, how="left")
    combined = combined.merge(market, left_index=True, right_index=True, how="left")  
    
    d = pd.DataFrame()
    for key_word in kw_list_2:
        geoff = pageviewapi.per_article('en.wikipedia', key_word, start, end,
                                    access='all-access', agent='all-agents', granularity='daily')
        dicty = dict(geoff)
        views = pd.DataFrame(dicty["items"])
        views["timestamp"] = pd.to_datetime((views["timestamp"]), format="%Y%m%d%H")
        views = views.set_index("timestamp")
        page = pd.Series(views["views"])
        d[key_word] = page
        
    combined = combined.merge(d, left_index=True, right_index=True, how="left") 
    
    return combined

In [65]:
Big_scraper(kw_list_1, kw_list_2,"F", "20190101", "20190107")

,Ford,F-150,Ford Bronco_x,Ford Mustang_x,Ford Stock,Open,High,Low,Close,Volume,...,nas_low,nas_close,nas_vol,Ford Motor Company,Ford Mustang_y,Ford F Series,Ford Bronco_y,Lincoln Navigator,Lincoln Aviator,Ford GT
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,13.565217,0.000000,0.000000,1.173913,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5163,3279,15,1372,668,329,1290
2019-01-02,18.791667,0.458333,0.000000,0.500000,0.000000,6.827570,7.271861,6.782234,7.163055,47494400.0,...,6506.879883,6665.939941,2.261800e+09,5648,3804,24,1597,710,341,1466
2019-01-03,22.166667,0.458333,1.000000,1.500000,0.416667,7.226524,7.244658,7.054248,7.054248,39172400.0,...,6457.129883,6463.500000,2.607290e+09,6290,3792,11,1549,678,334,1375
2019-01-04,18.333333,0.000000,0.458333,0.750000,1.125000,7.172122,7.362532,7.117719,7.326263,43039800.0,...,6554.240234,6738.859863,2.579550e+09,6227,3800,15,1387,679,303,1390
2019-01-05,23.291667,0.000000,0.625000,1.750000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6131,3872,22,3319,761,339,1490
2019-01-06,16.416667,0.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6778,3910,15,1587,734,445,1470
2019-01-07,20.708333,0.000000,1.291667,0.708333,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6110,3760,25,1507,727,444,1277


In [66]:
kw_list_1 = ["Ford", "F-150", "Ford Bronco", "Ford Mustang", "Ford Stock"]
kw_list_2 = ["Ford Motor Company", "Ford Mustang", "Ford F Series", "Ford Bronco", "Lincoln Navigator", "Lincoln Aviator", "Ford GT"]

In [68]:
Ford = Big_scraper(kw_list_1, kw_list_2,"F", "20190101", "20191231")

In [ ]:
Ford.to_csv("2019_Ford_Data.csv")

In [ ]:
Ford_20 = Big_scraper(kw_list_1, kw_list_2,"F", "20200101", "20201231")
Ford_20.to_csv("2020_Ford_Data.csv")

In [ ]:
Ford_21 = Big_scraper(kw_list_1, kw_list_2,"F", "20210101", "20211231")
Ford_21.to_csv("2021_Ford_Data.csv")

In [ ]:
Ford_22 = Big_scraper(kw_list_1, kw_list_2,"F", "20220101", "20220331")
Ford_22.to_csv("2022_Ford_Data.csv")

In [ ]:
ford = pd.concat([Ford, Ford_20, Ford_21, Ford_22])

In [70]:
dow = yf.Ticker("^DJI")
dow_h = dow.history(start="2019-01-01", end="2022-03-31")
dow_h = pd.DataFrame(dow_h)
dow_names = {"Open":"dow_open","Close":"dow_close","Low": "dow_low",
    'High':'dow_high','Volume':'dow_vol'}
dow_h=dow_h.rename(dow_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

nas = yf.Ticker("^IXIC")
nas_h = nas.history(start="2019-01-01", end="2022-03-31")
nas_h = pd.DataFrame(nas_h)
nas_names = {"Open":"nas_open", "Close":"nas_close", "Low": "nas_low",
    'High':'nas_high','Volume':'nas_vol'}
nas_h=nas_h.rename(nas_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

market = dow_h.merge(nas_h,left_index=True, right_index=True, how="left")

In [71]:
ford = ford.merge(market,left_index=True, right_index=True, how="left")

In [74]:
ford.head()

,Ford,F-150,Ford Bronco_x,Ford Mustang_x,Ford Stock,Open,High,Low,Close,Volume,...,dow_open,dow_high,dow_low,dow_close,dow_vol,nas_open,nas_high,nas_low,nas_close,nas_vol
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,14.043478,0.0,0.000000,1.217391,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,19.416667,0.5,0.000000,0.541667,0.000000,6.827570,7.271861,6.782234,7.163055,47494400.0,...,23058.609375,23413.470703,22928.589844,23346.240234,321570000.0,6506.910156,6693.709961,6506.879883,6665.939941,2.261800e+09
2019-01-03,23.041667,0.5,1.083333,1.500000,0.458333,7.226524,7.244659,7.054249,7.054249,39172400.0,...,23176.390625,23176.390625,22638.410156,22686.220703,424240000.0,6584.770020,6600.209961,6457.129883,6463.500000,2.607290e+09
2019-01-04,19.208333,0.0,0.500000,0.791667,1.208333,7.172122,7.362532,7.117719,7.326263,43039800.0,...,22894.919922,23518.640625,22894.919922,23433.160156,396020000.0,6567.140137,6760.689941,6554.240234,6738.859863,2.579550e+09
2019-01-05,24.000000,0.0,0.666667,1.875000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
ford.columns

Index(['Ford', 'F-150', 'Ford Bronco_x', 'Ford Mustang_x', 'Ford Stock',
       'Open', 'High', 'Low', 'Close', 'Volume',
       ...
       'dow_open', 'dow_high', 'dow_low', 'dow_close', 'dow_vol', 'nas_open',
       'nas_high', 'nas_low', 'nas_close', 'nas_vol'],
      dtype='object', length=110)

In [76]:
# creating base varibles to be used in variable creator functions
ford["Wiki_total"] = (ford["Ford Motor Company"] + 
    ford["Ford Mustang_y"] + ford["Ford F Series"] + 
    ford["Ford Bronco_y"] + ford["Lincoln Navigator"] + 
    ford["Lincoln Aviator"] + ford["Ford GT"])

ford["Google_total"] = (ford["Ford"] +
    ford["F-150"] + ford["Ford Bronco_x"] +
    ford["Ford Mustang_x"] + ford["Ford Stock"])

ford["Stock_total"] = ford["Close"]
ford["Nas_total"] = ford["nas_close"]
ford["Dow_total"] = ford["dow_close"]

In [77]:
def variables_creator(df, variable_list, w=7):
    
    '''
    descrition:
    -----------
    takes dataframe and returns new varibles based on recommmended calcualtions, 
    it should be done seporately with internet based and stock based dataframes

    Note: When using for aggreated varaibles, for example Wiki_total, the sum of all the wiki pages daily page view counts, 
    you MUST calculate Wiki_total in the dataframe seperately BEFORE this function can be used. 

    For example if you had wiki page counts for Ford Bronco and Ford Ranger, Wiki_total would equal Ford Bronco + Ford ranger counts, 
    AGAIN Wiki_total must be calculated in the desired dateframe before using this function 

    input:
    ------
    df: dataframe containing the google trends, yahoo finance, and or wikipedia page count data

    variable_list: list of strings to be added to the equations to calculate the new varaibles. 
    ex. insertting the string "Wiki" will add to df[f"{}_total"] to become "Wiki_total"

    w: the window length for one period shift. Default is 7 providing 7 day moving averages for wiki and google data, 
        FOR STOCK DATA THIS WILL NEED TO BE CHANGED TO 5.

    output:
    -------
    df: the same dataframe as was inputted but now containing variables for 
    Momemtum, Disparity, Moving Average, Exponential Moving Aerage, Rator Change, and RSI index score.
    Also containg are moving variables, which are boolean with 1 indicating an increase in the above variables
    '''   
    
    for i in variable_list:
        # Momentum_1
        df[f"{i}_Moment_1"] =  (df[f"{i}_total"] / df[f"{i}_total"].shift(w)) * 100
        # Momentum_2
        df[f"{i}_Moment_2"] =  (df[f"{i}_total"] - df[f"{i}_total"].shift(w)) * 100
        # Momentum_1_s three day shift (instead of w)
        df[f"{i}_Moment_1_s"] =  (df[f"{i}_total"] / df[f"{i}_total"].shift(3)) * 100
        # Momentum_2_s
        df[f"{i}_Moment_2_s"] =  (df[f"{i}_total"] - df[f"{i}_total"].shift(3)) * 100
        # Moving average
        df[f"{i}_MAvg"] = df[f"{i}_total"].rolling(f"{w}d").mean()
        # Moving average 3 day
        df[f"{i}_MAvg_s"] = df[f"{i}_total"].rolling("3d").mean()
        # Disparity
        df[f"{i}_Disparity"] = (df[f"{i}_total"]/df[f"{i}_MAvg"]) * 100
        # Disparity 3 day
        df[f"{i}_Disparity_s"] = (df[f"{i}_total"]/df[f"{i}_MAvg_s"]) * 100
        # Rate of Change Normal Way
        df[f"{i}_ROC"] = (df[f"{i}_total"]-df[f"{i}_total"].shift(w))/(df[f"{i}_total"].shift(w)) *100
        df[f"{i}_ROC_s"] = (df[f"{i}_total"]-df[f"{i}_total"].shift(3))/(df[f"{i}_total"].shift(3)) *100
        #Rate of Change Paper Way (doesn't make sense but just in case)
        df[f'{i}_Rocp'] = (df[f"{i}_total"]/df[f"{i}_Moment_2"]) *100
        # Exponential Moving Average
        df[f"{i}_EMA"] = (df[f"{i}_total"]-df[f"{i}_MAvg"].shift(1))*(2/(w+1))+df[f"{i}_MAvg"].shift(1)

        # calculating the Relative Strength Index, based on 14 day window
        df[f"{i}_diff"] = df[f"{i}_total"].diff(1)
        df[f"{i}_gain"] = df[f"{i}_diff"].clip(lower=0).round(2) #keeps all values above or below a given threshold, lower=lower bound
        df[f"{i}_loss"] = df[f"{i}_diff"].clip(upper=0).round(2)
        df[f'{i}_avg_gain'] = df[f'{i}_gain'].rolling(14).mean()
        df[f'{i}_avg_loss'] = df[f'{i}_loss'].rolling(14).mean()
        df[f'{i}_rs'] = df[f'{i}_avg_gain'] / df[f'{i}_avg_loss']
        df[f'{i}_RSI'] = 100 - (100 / (1.0 + df[f'{i}_rs']))

        # Calculatiing the Move Variables 
        df[f"{i}_Move"] = df[f"{i}_total"] > df[f"{i}_total"].shift(1) 
        df[f"{i}_Move"] = df[f"{i}_Move"].replace({True:1,False: 0})
    
      
        df[f"{i}_MAvg_Move"] = df[f"{i}_MAvg"] > df[f"{i}_MAvg"].shift(1) 
        df[f"{i}_MAvg_Move"] = df[f"{i}_MAvg_Move"].replace({True:1,False: 0})
        df[f"{i}_MAvg_s_Move"] = df[f"{i}_MAvg_s"] > df[f"{i}_MAvg_s"].shift(1) 
        df[f"{i}_MAvg_s_Move"] = df[f"{i}_MAvg_s_Move"].replace({True:1,False: 0})

        df[f"{i}_EMA_Move"] = df[f"{i}_EMA"] > df[f"{i}_EMA"].shift(1) 
        df[f"{i}_EMA_Move"] = df[f"{i}_EMA_Move"].replace({True:1,False: 0})

        df[f"{i}_Disparity_Move"] = df[f"{i}_Disparity"] > df[f"{i}_Disparity"].shift(1) 
        df[f"{i}_Disparity_Move"] = df[f"{i}_Disparity_Move"].replace({True:1,False: 0})
        df[f"{i}_Disparity_s_Move"] = df[f"{i}_Disparity_s"] > df[f"{i}_Disparity_s"].shift(1) 
        df[f"{i}_Disparity_s_Move"] = df[f"{i}_Disparity_s_Move"].replace({True:1,False: 0})

        df[f"{i}_RSI_Move"] = df[f"{i}_RSI"] > df[f"{i}_RSI"].shift(1) 
        df[f"{i}_RSI_Move"] = df[f"{i}_RSI_Move"].replace({True:1,False: 0})
        
    return df

In [79]:
ford = variables_creator(ford, ["Wiki","Google", "Stock", "Nas", "Dow"])

In [89]:
def target_creator(df):
    '''
    description: creates the differnt types of target variables based on tomorrow minus today,
    '''
        
    # target 1, Open(t+1) - Close(t)
    df["target_1"] = (df["Open"].shift(-1) - df["Close"]) > 0
    df["target_1"] = df["target_1"].replace({True:1,False: 0})
    # target 2
    df["target_2"] = (df["Open"].shift(-1) - df["Open"]) > 0
    df["target_2"] = df["target_2"].replace({True:1,False: 0})
    # target 3
    df["target_3"] = (df["Close"].shift(-1) - df["Close"]) > 0
    df["target_3"] = df["target_3"].replace({True:1,False: 0})
    # target 4
    df["target_4"] = (df["Close"].shift(-1) - df["Open"]) > 0
    df["target_4"] = df["target_4"].replace({True:1,False: 0})
    # target 5
    df["target_5"] = (df["Volume"].shift(-1) - df["Volume"]) > 0
    df["target_5"] = df["target_5"].replace({True:1,False: 0})
 
    return df

In [90]:
ford = target_creator(ford)

In [91]:
ford.to_csv("Ford_full_data.csv")